In [1]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import decomposition
from matplotlib import pyplot as plt
from sklearn import neighbors

from repeated_CV_builder import DCV

In [3]:
# imports the data
data = pd.read_csv("raw_data/train_call.txt", index_col=0, delimiter="\t").transpose()
topInfo = data.iloc[:3]
data = data.iloc[4:].dropna(axis=1)
classes = pd.read_csv("raw_data/train_clinical.txt", delimiter="\t", index_col=0)
data = classes.join(data).set_index("Subgroup").dropna()
# Add 1 to data because of chi-square feature selection


DCV.outer_repeats = 5
DCV.inner_repeats = 10
DCV.num_features = 150

model = neighbors.KNeighborsClassifier(n_jobs=-1)
modeller = DCV(model)
# prints all the params you can change :)
print(model.get_params().keys())

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])


In [4]:
# select which params you want to test with the inner loop
# space is the paramater space
modeller.hyperParams['n_neighbors'] = range(3,21,2)
#modeller.hyperParams['l1_ratio'] = [0.1, 0.3, 0.5, 0.7, 0.9]


modeller.hyperParams

{'n_neighbors': range(3, 21, 2)}

In [10]:
# this does everything for you :)
# first is the data used, seconds comes the classifications and than with loop the amount of loops you want to do

modeller.train_fit(data=data, classes=data.index, loop=100)
# some test code

print(accuracy_score(DCV.__class_cleaner__(None, data.index),modeller.predict(data)))

Training progress: [--------------------------------------------------] 0.0% 
Training progress: [--------------------------------------------------] 0.2% 
Training progress: [--------------------------------------------------] 0.4% 
Training progress: [--------------------------------------------------] 0.6% 
Training progress: [--------------------------------------------------] 0.8% 
Training progress: [--------------------------------------------------] 1.0% 
Training progress: [#-------------------------------------------------] 1.2% 
Training progress: [#-------------------------------------------------] 1.4% 
Training progress: [#-------------------------------------------------] 1.6% 
Training progress: [#-------------------------------------------------] 1.8% 
Training progress: [#-------------------------------------------------] 2.0% 
Training progress: [#-------------------------------------------------] 2.2% 
Training progress: [#-------------------------------------------

Training progress: [##########----------------------------------------] 21.0% 
Training progress: [###########---------------------------------------] 21.2% 
Training progress: [###########---------------------------------------] 21.4% 
Training progress: [###########---------------------------------------] 21.6% 
Training progress: [###########---------------------------------------] 21.8% 
Training progress: [###########---------------------------------------] 22.0% 
Training progress: [###########---------------------------------------] 22.2% 
Training progress: [###########---------------------------------------] 22.4% 
Training progress: [###########---------------------------------------] 22.6% 
Training progress: [###########---------------------------------------] 22.8% 
Training progress: [############--------------------------------------] 23.0% 
Training progress: [############--------------------------------------] 23.2% 
Training progress: [############--------------------

Training progress: [#####################-----------------------------] 41.8% 
Training progress: [#####################-----------------------------] 42.0% 
Training progress: [#####################-----------------------------] 42.2% 
Training progress: [#####################-----------------------------] 42.4% 
Training progress: [#####################-----------------------------] 42.6% 
Training progress: [#####################-----------------------------] 42.8% 
Training progress: [######################----------------------------] 43.0% 
Training progress: [######################----------------------------] 43.2% 
Training progress: [######################----------------------------] 43.4% 
Training progress: [######################----------------------------] 43.6% 
Training progress: [######################----------------------------] 43.8% 
Training progress: [######################----------------------------] 44.0% 
Training progress: [######################----------

Training progress: [###############################-------------------] 62.6% 
Training progress: [###############################-------------------] 62.8% 
Training progress: [################################------------------] 63.0% 
Training progress: [################################------------------] 63.2% 
Training progress: [################################------------------] 63.4% 
Training progress: [################################------------------] 63.6% 
Training progress: [################################------------------] 63.8% 
Training progress: [################################------------------] 64.0% 
Training progress: [################################------------------] 64.2% 
Training progress: [################################------------------] 64.4% 
Training progress: [################################------------------] 64.6% 
Training progress: [################################------------------] 64.8% 
Training progress: [################################

Training progress: [##########################################--------] 83.4% 
Training progress: [##########################################--------] 83.6% 
Training progress: [##########################################--------] 83.8% 
Training progress: [##########################################--------] 84.0% 
Training progress: [##########################################--------] 84.2% 
Training progress: [##########################################--------] 84.4% 
Training progress: [##########################################--------] 84.6% 
Training progress: [##########################################--------] 84.8% 
Training progress: [##########################################--------] 85.0% 
Training progress: [###########################################-------] 85.2% 
Training progress: [###########################################-------] 85.4% 
Training progress: [###########################################-------] 85.6% 
Training progress: [################################

IndexError: positional indexers are out-of-bounds

In [32]:
pd.DataFrame([x[0] for x in modeller.all_features],columns=range(len(topInfo.transpose())))

ValueError: 2834 columns passed, passed data had 150 columns

In [16]:
print(topInfo.transpose().iloc[modeller.get_best_features()])

IndexError: positional indexers are out-of-bounds

In [17]:
modeller.test_accuracy.groupby(["n_neighbors"])["values"].describe()

,count,mean,std,min,25%,50%,75%,max
n_neighbors,,,,,,,,
3,113.0,0.460619,0.098301,0.25,0.40,0.45,0.5000,0.75
5,123.0,0.483740,0.108359,0.30,0.40,0.50,0.5500,0.85
7,94.0,0.474468,0.104918,0.30,0.40,0.45,0.5375,0.85
9,40.0,0.463750,0.107410,0.30,0.35,0.45,0.5500,0.75
11,41.0,0.501220,0.099050,0.30,0.45,0.50,0.5500,0.70
13,27.0,0.468519,0.107550,0.30,0.40,0.45,0.5500,0.65
15,17.0,0.508824,0.061835,0.40,0.45,0.50,0.5500,0.60
17,23.0,0.482609,0.104020,0.35,0.40,0.50,0.5500,0.70
19,22.0,0.461364,0.110121,0.30,0.40,0.45,0.5375,0.65


In [ ]:
modeller.test_precision

In [ ]:
modeller.test_recall


In [18]:
cm = confusion_matrix(data.index, modeller.predict(data))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=modeller.get_best_model().classes_)
disp.plot()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
modeller.get_best_model()

In [ ]:
import scipy.stats as stats
model.predict_proba(data,)
stats.wilcoxon(modeller.get_best_model.predict_proba(data), data.index)

In [ ]:
print(modeller.get_best_features())

#todo
box plots accuracy, precision, recall
best model. and params features

save all the juicy info (params, and features)

In [ ]:
modeller.all_accuracies
#modeller.all_precision
#modeller.all_recall

In [19]:
import math
print("accuracy:",modeller.test_accuracy.groupby(["n_neighbors"])["values"].mean().loc[7])
print("accuracy:",modeller.test_accuracy.groupby(["n_neighbors"])["values"].mean().loc[5])
print("precision:",modeller.test_precision.groupby(["n_neighbors"])["values"].mean().loc[7])
print("precision:",modeller.test_precision.groupby(["n_neighbors"])["values"].mean().loc[5])
print("recall:",modeller.test_recall.groupby(["n_neighbors"])["values"].mean().loc[7])
print("recall:",modeller.test_recall.groupby(["n_neighbors"])["values"].mean().loc[5])
ci95_hi = []
ci95_lo = []
stats = modeller.test_accuracy.groupby(["n_neighbors"])["values"].agg(['mean', 'count', 'std'])
for i, x in enumerate(stats.index):
    #print(i)
    m, c, s = stats.iloc[i]
    #print(m, c, s)
    ci95_hi += [m + 1.95*s/math.sqrt(c)]
    ci95_lo += [m - 1.95*s/math.sqrt(c)]
stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.loc[5])
print(stats.loc[7])


print()
print(pd.DataFrame([["",7,0.56,0.58,0.56,"0.54-0.57"],["",5,0.55,0.57,0.56,"0.54-0.57"]]


,columns=["","Nr neighbors", "Mean accuracy",
          "Mean precision", "Mean recall", "Confidence interval"]).to_latex(
    index=False,
    caption=("full_caption","short_caption"),
    col_space = [1/6 for x in range(6)]
))

accuracy: 0.47446808510638305
accuracy: 0.4837398373983741
precision: 0.43366538543527877
precision: 0.4629640364147902
recall: 0.4725599459642012
recall: 0.48477222867466774
mean         0.483740
count      123.000000
std          0.108359
ci95_hi      0.502792
ci95_lo      0.464688
Name: 5, dtype: float64
mean        0.474468
count      94.000000
std         0.104918
ci95_hi     0.495570
ci95_lo     0.453366
Name: 7, dtype: float64

\begin{table}
\centering
\caption[short_caption]{full_caption}
\begin{tabular}{lrrrrl}
\toprule
{} &  Nr neighbors &  Mean accuracy &  Mean precision &  Mean recall & Confidence interval \\
\midrule
{} &             7 &           0.56 &            0.58 &         0.56 &           0.54-0.57 \\
{} &             5 &           0.55 &            0.57 &         0.56 &           0.54-0.57 \\
\bottomrule
\end{tabular}
\end{table}



In [24]:
modelName = "KNN"
modeller.test_accuracy.to_csv("{}_test_accuracy.csv".format(modelName))
modeller.test_precision.to_csv("{}_test_precision.csv".format(modelName))
modeller.test_recall.to_csv("{}_test_recall.csv".format(modelName))
modeller.train_accuracies.to_csv("{}_train_accuracy.csv".format(modelName))
modeller.train_precision.to_csv("{}_train_precision.csv".format(modelName))
modeller.train_recall.to_csv("{}_train_recall.csv".format(modelName))
pd.DataFrame(modeller._best_features).to_csv("{}_features.csv".format(modelName))
pd.DataFrame(modeller._best_params).to_csv("{}_features.csv".format(modelName))
